# Import

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
!pip install sentence_transformers

In [17]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import random

from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader

from scipy import spatial
import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error 

import gensim
from nltk.tokenize import WordPunctTokenizer
import tensorflow as tf
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Embedding

from tensorflow.python.keras.models import Model, Layer, Sequential
from tensorflow.python.keras.layers import LSTM, Bidirectional, Subtract, Input, Dense, Flatten, Dot
import keras.backend as K

In [18]:
%notebook inline

In [19]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

# Constant

In [20]:
ROOT_PATH = '/content/drive/My Drive/13516085 - Muh Habibi Haidir';

# Helper Function

In [21]:
def changes_data_to_x(tokenizer, data_train, data_test):
    sequences_train_student = t.texts_to_sequences(data_train['jawaban_pelajar'])
    sequences_train_reference = t.texts_to_sequences(data_train['jawaban_referensi'])
    sequences_test_student = t.texts_to_sequences(data_test['jawaban_pelajar'])
    sequences_test_reference = t.texts_to_sequences(data_test['jawaban_referensi'])

    X_train_student = pad_sequences(sequences_train_student, maxlen=MAX_SEQUENCE_LENGTH)
    X_train_reference = pad_sequences(sequences_train_reference, maxlen=MAX_SEQUENCE_LENGTH)
    X_test_student = pad_sequences(sequences_test_student, maxlen=MAX_SEQUENCE_LENGTH)
    X_test_reference = pad_sequences(sequences_test_reference, maxlen=MAX_SEQUENCE_LENGTH)
    
    return X_train_student, X_train_reference, X_test_student, X_test_reference

# Main

## Prepare Data

In [56]:
student_saintek_answer_train = pd.read_csv('{}/Code/Data/Indonesia/score-saintek.csv'.format(ROOT_PATH))
total_problem_saintek_train = len(np.unique(student_saintek_answer_train['type-problem']))

student_soshum_answer_train = pd.read_csv('{}/Code/Data/Indonesia/score-soshum.csv'.format(ROOT_PATH))
student_soshum_answer_train['type-problem'] = student_soshum_answer_train['type-problem'] + total_problem_saintek_train

question_saintek_answer_train = pd.read_csv('{}/Code/Data/Indonesia/question-saintek.csv'.format(ROOT_PATH))
question_soshum_answer_train = pd.read_csv('{}/Code/Data/Indonesia/question-soshum.csv'.format(ROOT_PATH))
question_soshum_answer_train['type-problem'] = question_soshum_answer_train['type-problem'] + total_problem_saintek_train

student_saintek_answer_test = pd.read_csv('{}/Code/Data/Indonesia/score-saintek-test.csv'.format(ROOT_PATH))
total_problem_saintek_test = len(np.unique(student_saintek_answer_test['type-problem']))

student_soshum_answer_test = pd.read_csv('{}/Code/Data/Indonesia/score-soshum-test.csv'.format(ROOT_PATH))
student_soshum_answer_test['type-problem'] = student_soshum_answer_test['type-problem'] + total_problem_saintek_test

question_saintek_answer_test = pd.read_csv('{}/Code/Data/Indonesia/question-saintek-test.csv'.format(ROOT_PATH))
question_soshum_answer_test = pd.read_csv('{}/Code/Data/Indonesia/question-soshum-test.csv'.format(ROOT_PATH))
question_soshum_answer_test['type-problem'] = question_soshum_answer_test['type-problem'] + total_problem_saintek_test

In [57]:
student_answer_train = pd.concat([student_saintek_answer_train, student_soshum_answer_train])
question_answer_train = pd.concat([question_saintek_answer_train, question_soshum_answer_train])

student_answer_test = pd.concat([student_saintek_answer_test, student_soshum_answer_test])
question_answer_test = pd.concat([question_saintek_answer_test, question_soshum_answer_test])

print('Student Answer Train Shape: {}'.format(student_answer_train.shape))
display(student_answer_train.head())
print('Question Answer Train Shape: {}'.format(question_answer_train.shape))
display(question_answer_train.head())

print('Student Answer Test Shape: {}'.format(student_answer_test.shape))
print('Question Answer Test Shape: {}'.format(question_answer_test.shape))

Student Answer Train Shape: (7526, 3)


,type-problem,jawaban,score
0,1,Bakteri penyubur tanaman,1
1,1,"Entahlah, yang jelas bakteri berplasmid",1
2,1,Bakteri baik Bakteri hasil mutasi.,1
3,1,Bakteri,0
4,1,"Pseudomonas syringae, agrobacterium tumefacien...",4


Question Answer Train Shape: (30, 3)


,type-problem,pertanyaan,jawaban
0,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
1,2,Jelaskan perbedaan antara meristem primer deng...,Meristem primer terlibat dalam pertumbuhan pri...
2,3,Apa yang dimaksud dengan masa dormansi biji? A...,Dormansi biji adalah keadaan ketika biji mence...
3,4,Apakah yang dimaksud perkembangan?,Perkembangan adalah transformasi suatu orgnai...
4,5,Di daerah diferensiasi terdapat tiga sistem ja...,Jaringan dermal adalah perlindungan mencegah k...


Student Answer Test Shape: (1560, 3)
Question Answer Test Shape: (6, 3)


In [58]:
idx_problem_train_saintek = sklearn.utils.shuffle(np.arange(1, 16), random_state=21)
idx_problem_train_soshum = sklearn.utils.shuffle(np.arange(16, 31), random_state=21)

idx_problem_train = np.concatenate((idx_problem_train_saintek[3:], idx_problem_train_soshum[3:]))
idx_problem_val = np.concatenate((idx_problem_train_saintek[:3], idx_problem_train_soshum[:3])) 

In [59]:
data_train = pd.merge(student_answer_train, question_answer_train, how='inner', on='type-problem', suffixes=(['_pelajar', '_referensi']))
data_train = data_train.fillna('')
data_train = data_train[data_train['score']!=0]
data_test = pd.merge(student_answer_test, question_answer_test, how='inner', on='type-problem', suffixes=(['_pelajar', '_referensi']))
data_test = data_test.fillna('')
data_test = data_test[data_test['score']!=0]

data_val = data_train[data_train['type-problem'].isin(idx_problem_val)]
data_train = data_train[data_train['type-problem'].isin(idx_problem_train)]

y_test = data_test['score']/5
y_train = data_train['score']/5
y_val = data_val['score']/5

print('Data Train Shape: {}'.format(data_train.shape))
display(data_train.head())
print('Data Val Shape: {}'.format(data_val.shape))
display(data_val.head())
print('Data Test Shape: {}'.format(data_test.shape))
display(data_test.head())

Data Train Shape: (5037, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
0,1,Bakteri penyubur tanaman,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
1,1,"Entahlah, yang jelas bakteri berplasmid",1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
2,1,Bakteri baik Bakteri hasil mutasi.,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
4,1,"Pseudomonas syringae, agrobacterium tumefacien...",4,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
5,1,"Thiobacillus ferooxidans, Agrobacterium tumefa...",4,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"


Data Val Shape: (1261, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
1116,6,Sel spons,1,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1120,6,"Dermatogen, periblem, plerom",4,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1121,6,"Dermatogen, lapisan yang akan membentuk epider...",5,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1122,6,"dermatogen, periblem, plerom",4,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1125,6,"Endoderm, mesoderm, ektoderm",1,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."


Data Test Shape: (1406, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
0,1,Ekosistem adalah kumpulan materi abiotik dan b...,3,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
1,1,suatu sistem ekologi yang terbentuk oleh hubun...,3,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
2,1,ekosistem adalah komunitas organisme makhluk h...,5,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
3,1,Ekosistem adalah satu kesatuan makhluk hidup d...,2,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
4,1,Hubungan timbal balik antara mahkluk hidup den...,5,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...


## Experiment

### Constant

In [26]:
NUM_EPOCH = 100
BATCH_SIZE = 10

### Main

#### Sentence Transformer

##### Linear Regression

In [34]:
models_name = ['{}/Code/Model/model-bert-indonesia/model-distilbert-base-multilingual-cased-max'.format(ROOT_PATH)]
input_types = ['sub']

In [61]:
for model_name in models_name:
  print('Model Ongoing: {}'.format(model_name))

  sentence_model = SentenceTransformer(model_name)

  print('Answer Student Train Embedding...')
  data_train['answer_student_embed'] = sentence_model.encode(data_train['jawaban_pelajar'].str.lower().values);
  print('Answer Reference Train Embedding...')
  data_train['answer_reference_embed'] = sentence_model.encode(data_train['jawaban_referensi'].str.lower().values);
  print('Answer Student Test Embedding...')
  data_test['answer_student_embed'] = sentence_model.encode(data_test['jawaban_pelajar'].str.lower().values);
  print('Answer Reference Test Embedding...')
  data_test['answer_reference_embed'] = sentence_model.encode(data_test['jawaban_referensi'].str.lower().values);

  y_train = data_train['score']/5
  y_test = data_test['score']/5

  for input_type in input_types:
    if input_type == 'nothing':
      siamese_bert_model = tf.keras.Sequential()
      siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*2)))
      siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

      train_s1 = data_train['answer_student_embed'].values
      train_s1 = np.array([np.array(s1) for s1 in train_s1])
      train_s2 = data_train['answer_reference_embed'].values
      train_s2 = np.array([np.array(s2) for s2 in train_s2])
      X_train = np.concatenate((train_s1, train_s2), axis=1)

      test_s1 = data_test['answer_student_embed'].values
      test_s1 = np.array([np.array(s1) for s1 in test_s1])
      test_s2 = data_test['answer_reference_embed'].values
      test_s2 = np.array([np.array(s2) for s2 in test_s2])
      X_test = np.concatenate((test_s1, test_s2), axis=1)
    elif input_type == 'sub':
      siamese_bert_model = tf.keras.Sequential()
      siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*3)))
      siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

      test_s1 = data_test['answer_student_embed'].values
      test_s1 = np.array([np.array(s1) for s1 in test_s1])
      test_s2 = data_test['answer_reference_embed'].values
      test_s2 = np.array([np.array(s2) for s2 in test_s2])
      X_test = np.concatenate((test_s1, test_s2, np.subtract(test_s1, test_s2)), axis=1)
    else:
      siamese_bert_model = tf.keras.Sequential()
      siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*4)))
      siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

      train_s1 = data_train['answer_student_embed'].values
      train_s1 = np.array([np.array(s1) for s1 in train_s1])
      train_s2 = data_train['answer_reference_embed'].values
      train_s2 = np.array([np.array(s2) for s2 in train_s2])
      X_train = np.concatenate((train_s1, train_s2, np.subtract(train_s1, train_s2), np.multiply(train_s1, train_s2)), axis=1)

      test_s1 = data_test['answer_student_embed'].values
      test_s1 = np.array([np.array(s1) for s1 in test_s1])
      test_s2 = data_test['answer_reference_embed'].values
      test_s2 = np.array([np.array(s2) for s2 in test_s2])
      X_test = np.concatenate((test_s1, test_s2, np.subtract(test_s1, test_s2), np.multiply(test_s1, test_s2)), axis=1)


    checkpoint_path = '{}/Code/Model/model-indonesia/sentence_embedding/indonesia.ckpt'.format(ROOT_PATH)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    verbose=1)
  

    history = siamese_bert_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_data=(X_test, y_test), verbose=0, callbacks=[cp_callback])
    # siamese_bert_model.load_weights(checkpoint_path)
    predicts_test = siamese_bert_model.predict(X_test)
    predicts_train = siamese_bert_model.predict(X_train)

    print('Model Name: {}, Pooling Type: {}, Input Type: {} Result:'.format(model_name, pooling_type, input_type))
    print('Train Mean Absolute Error: {}'.format(round(mean_absolute_error(y_train, predicts_train ), 4)))
    print('Train Mean Squared Error: {}'.format(round(mean_squared_error(y_train, predicts_train), 4)))
    print('Val Mean Absolute Error: {}'.format(round(mean_absolute_error(y_test, predicts_test), 4)))
    print('Val Mean Squared Error: {}'.format(round(mean_squared_error(y_test, predicts_test), 4)))  
    
  del sentence_model

Model Ongoing: /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-bert-indonesia/model-distilbert-base-multilingual-cased-max
Answer Student Train Embedding...
Answer Reference Train Embedding...
Answer Student Test Embedding...
Answer Reference Test Embedding...

Epoch 00001: saving model to /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-indonesia/sentence_embedding/indonesia.ckpt

Epoch 00002: saving model to /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-indonesia/sentence_embedding/indonesia.ckpt

Epoch 00003: saving model to /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-indonesia/sentence_embedding/indonesia.ckpt

Epoch 00004: saving model to /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-indonesia/sentence_embedding/indonesia.ckpt

Epoch 00005: saving model to /content/drive/My Drive/13516085 - Muh Habibi Haidir/Code/Model/model-indonesia/sentence_embedding/indonesia.ck

In [ ]:
test_df = data_test[['type-problem', 'pertanyaan', 'jawaban_referensi',  'jawaban_pelajar', 'score']]
test_df['predict'] = [predict_test[0]*5 for predict_test in predicts_test]
# test_df = test_df[test_df['score'] != 0]
test_df['error'] = np.abs(test_df['score'] - test_df['predict'])

In [73]:
with open('{}/data-answers.txt'.format(ROOT_PATH), 'w') as data_answers_file:
  for idx, row in test_df.sort_values(['error']).iterrows():
    jawaban_pelajar = row['jawaban_pelajar']
    jawaban_referensi = row['jawaban_referensi']
    pertanyaan = row['pertanyaan']
    error = row['error']
    score = row['score']
    predict = row['predict']

    data_answers_file.write(pertanyaan + '\n')
    data_answers_file.write(jawaban_pelajar + '\n')
    data_answers_file.write(jawaban_referensi + '\n')
    data_answers_file.write('{}, {}, {}\n\n'.format(score, predict, error))

In [75]:
test_df[['type-problem', 'pertanyaan']].drop_duplicates()

,type-problem,pertanyaan
0,1,Apakah yang dimaksud dengan ekosistem?
260,2,Apa perbedaan jaringan makanan dan rantai maka...
520,3,Sebutkan komponen biotik dari ekosistem?
780,4,Apa pengertian dari geografi menurut Ikatan Ge...
1040,5,Sebutkan susunan lapisan litosfer!
1300,6,Jelaskan teori apungan benua yang dikemukakan ...


In [74]:
test_df.groupby(['type-problem']).mean()

,score,predict,error
type-problem,,,
1,3.162791,3.438168,1.241317
2,3.644898,4.061285,0.874714
3,4.035433,4.208109,0.637068
4,3.158798,3.976594,0.966097
5,1.780269,2.255288,0.853426
6,2.233161,0.923092,1.349585


#### Transformer

##### Linear Regression

In [31]:
# models_name = ['distilbert-base-multilingual-cased', 'bert-base-multilingual-cased']
# pooling_types = ['mean-pooling', 'max-pooling', 'wkpooling']
# input_types = ['nothing', 'sub', 'multiply', 'cosine']
models_name = ['bert-base-multilingual-uncased']
pooling_types = ['max-pooling']
input_types = ['sub']

In [30]:
for model_name in models_name:
  print('Model Ongoing: {}'.format(model_name))

  word_embedding_model = models.Transformer(model_name)

  for pooling_type in pooling_types:
    if pooling_type == 'max-pooling':
      pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                    pooling_mode_mean_tokens=False,
                                    pooling_mode_cls_token=False,
                                    pooling_mode_max_tokens=True)
    elif pooling_type == 'mean-pooling':
      pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                    pooling_mode_mean_tokens=True,
                                    pooling_mode_cls_token=False,
                                    pooling_mode_max_tokens=False)
    else:
      word_embedding_model =  word_embedding_model = models.Transformer(model_name, model_args={'output_hidden_states': True})
      pooling_model = models.WKPooling(word_embedding_model.get_word_embedding_dimension())

    sentence_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    print('Answer Student Train Embedding...')
    data_train['answer_student_embed'] = sentence_model.encode(data_train['jawaban_pelajar'].str.lower().values);
    print('Answer Reference Train Embedding...')
    data_train['answer_reference_embed'] = sentence_model.encode(data_train['jawaban_referensi'].str.lower().values);
    print('Answer Student Val Embedding...')
    data_val['answer_student_embed'] = sentence_model.encode(data_val['jawaban_pelajar'].str.lower().values);
    print('Answer Reference Val Embedding...')
    data_val['answer_reference_embed'] = sentence_model.encode(data_val['jawaban_referensi'].str.lower().values);
    print('Answer Student Test Embedding...')
    data_test['answer_student_embed'] = sentence_model.encode(data_test['jawaban_pelajar'].str.lower().values);
    print('Answer Reference Test Embedding...')
    data_test['answer_reference_embed'] = sentence_model.encode(data_test['jawaban_referensi'].str.lower().values);

    y_train = data_train['score']/5
    y_test = data_test['score']/5
    y_val = data_val['score']/5

    train_s1 = data_train['answer_student_embed'].values
    train_s1 = np.array([np.array(s1) for s1 in train_s1])
    train_s2 = data_train['answer_reference_embed'].values
    train_s2 = np.array([np.array(s2) for s2 in train_s2])

    val_s1 = data_val['answer_student_embed'].values
    val_s1 = np.array([np.array(s1) for s1 in val_s1])
    val_s2 = data_val['answer_reference_embed'].values
    val_s2 = np.array([np.array(s2) for s2 in val_s2])

    test_s1 = data_test['answer_student_embed'].values
    test_s1 = np.array([np.array(s1) for s1 in test_s1])
    test_s2 = data_test['answer_reference_embed'].values
    test_s2 = np.array([np.array(s2) for s2 in test_s2])

    for input_type in input_types:
      if input_type == 'nothing':
        siamese_bert_model = tf.keras.Sequential()
        siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*2)))
        siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

        X_train = np.concatenate((train_s1, train_s2), axis=1)
        X_val = np.concatenate((val_s1, val_s2), axis=1)
        X_test = np.concatenate((test_s1, test_s2), axis=1)
      elif input_type == 'sub':
        siamese_bert_model = tf.keras.Sequential()
        siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*3)))
        siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

        X_train = np.concatenate((train_s1, train_s2, np.subtract(train_s1, train_s2)), axis=1)
        X_val = np.concatenate((val_s1, val_s2, np.subtract(val_s1, val_s2)), axis=1)
        X_test = np.concatenate((test_s1, test_s2, np.subtract(test_s1, test_s2)), axis=1)
      elif input_type == 'multiply':
        siamese_bert_model = tf.keras.Sequential()
        siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(sentence_model.get_sentence_embedding_dimension()*4)))
        siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

        X_train = np.concatenate((train_s1, train_s2, np.subtract(train_s1, train_s2), np.multiply(train_s1, train_s2)), axis=1)
        X_val = np.concatenate((val_s1, val_s2, np.subtract(val_s1, val_s2), np.multiply(val_s1, val_s2)), axis=1)
        X_test = np.concatenate((test_s1, test_s2, np.subtract(test_s1, test_s2), np.multiply(test_s1, test_s2)), axis=1)
      else:
        siamese_bert_model = tf.keras.Sequential()
        siamese_bert_model.add(Dense(1, activation='sigmoid', input_dim=(2)))
        siamese_bert_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

        trains_similarity = [1-spatial.distance.cosine(answer_student_embed, answer_reference_embed) for answer_student_embed, answer_reference_embed in zip(data_train['answer_student_embed'].values, data_train['answer_reference_embed'].values)]
        X_train = np.array([[1, train_similarity] for train_similarity in trains_similarity])
        vals_similarity = [1-spatial.distance.cosine(answer_student_embed, answer_reference_embed) for answer_student_embed, answer_reference_embed in zip(data_val['answer_student_embed'].values, data_val['answer_reference_embed'].values)]
        X_val = np.array([[1, val_similarity] for val_similarity in vals_similarity])
        tests_similarity = [1-spatial.distance.cosine(answer_student_embed, answer_reference_embed) for answer_student_embed, answer_reference_embed in zip(data_test['answer_student_embed'].values, data_test['answer_reference_embed'].values)]
        X_test = np.array([[1, test_similarity] for test_similarity in tests_similarity])

      print('Model Name: {}, Pooling Type: {}, Input Type: {} Result:'.format(model_name, pooling_type, input_type))
      history = siamese_bert_model.fit(X_train, y_train, batch_size=10, epochs=15, validation_data=(X_val, y_val), verbose=0)
      predicts_test = siamese_bert_model.predict(X_test)
      predicts_val = siamese_bert_model.predict(X_val)
      predicts_train = siamese_bert_model.predict(X_train)

      print('Train Mean Absolute Error: {}'.format(round(mean_absolute_error(y_train, predicts_train ), 4)))
      print('Train Mean Squared Error: {}'.format(round(mean_squared_error(y_train, predicts_train), 4)))
      print('Val Mean Absolute Error: {}'.format(round(mean_absolute_error(y_val, predicts_val), 4)))
      print('Val Mean Squared Error: {}'.format(round(mean_squared_error(y_val, predicts_val), 4)))
      print('Test Mean Absolute Error: {}'.format(round(mean_absolute_error(y_test, predicts_test), 4)))
      print('Test Mean Squared Error: {}'.format(round(mean_squared_error(y_test, predicts_test), 4)))  
      
    del sentence_model

Model Ongoing: distilbert-base-multilingual-cased



Answer Student Train Embedding...
Answer Reference Train Embedding...
Answer Student Val Embedding...
Answer Reference Val Embedding...
Answer Student Test Embedding...
Answer Reference Test Embedding...
Model Name: distilbert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: nothing Result:
Train Mean Absolute Error: 0.1379
Train Mean Squared Error: 0.0348
Val Mean Absolute Error: 0.3146
Val Mean Squared Error: 0.1551
Test Mean Absolute Error: 0.2642
Test Mean Squared Error: 0.1132
Model Name: distilbert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: sub Result:
Train Mean Absolute Error: 0.1307
Train Mean Squared Error: 0.0319
Val Mean Absolute Error: 0.2568
Val Mean Squared Error: 0.1124
Test Mean Absolute Error: 0.2214
Test Mean Squared Error: 0.0814
Model Name: distilbert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: multiply Result:
Train Mean Absolute Error: 0.1189
Train Mean Squared Error: 0.0274
Val Mean Absolute Error: 0.23


Answer Student Train Embedding...
Answer Reference Train Embedding...
Answer Student Val Embedding...
Answer Reference Val Embedding...
Answer Student Test Embedding...
Answer Reference Test Embedding...
Model Name: bert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: nothing Result:
Train Mean Absolute Error: 0.1312
Train Mean Squared Error: 0.032
Val Mean Absolute Error: 0.3723
Val Mean Squared Error: 0.2189
Test Mean Absolute Error: 0.3352
Test Mean Squared Error: 0.1819
Model Name: bert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: sub Result:
Train Mean Absolute Error: 0.1394
Train Mean Squared Error: 0.0352
Val Mean Absolute Error: 0.2947
Val Mean Squared Error: 0.138
Test Mean Absolute Error: 0.3417
Test Mean Squared Error: 0.185
Model Name: bert-base-multilingual-cased, Pooling Type: mean-pooling, Input Type: multiply Result:
Train Mean Absolute Error: 0.1293
Train Mean Squared Error: 0.0306
Val Mean Absolute Error: 0.2681
Val Mean Squared E

##### Cosine Similarity

In [ ]:
models_name = ['bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'distilbert-base-multilingual-cased']
pooling_types = ['mean-pooling', 'max-pooling', 'wkpooling']

In [ ]:
for model_name in models_name:
  print('Model Ongoing: {}'.format(model_name))

  word_embedding_model = models.Transformer(model_name)

  for pooling_type in pooling_types:
    if pooling_type == 'max-pooling':
      pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                    pooling_mode_mean_tokens=False,
                                    pooling_mode_cls_token=False,
                                    pooling_mode_max_tokens=True)
    elif pooling_type == 'mean-pooling':
      pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                    pooling_mode_mean_tokens=True,
                                    pooling_mode_cls_token=False,
                                    pooling_mode_max_tokens=False)
    else:
      word_embedding_model =  word_embedding_model = models.Transformer(model_name, model_args={'output_hidden_states': True})
      pooling_model = models.WKPooling(word_embedding_model.get_word_embedding_dimension())

    sentence_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    print('Answer Student Test Embedding...')
    data_test['answer_student_embed'] = sentence_model.encode(data_test['jawaban_pelajar'].str.lower().values);
    print('Answer Reference Test Embedding...')
    data_test['answer_reference_embed'] = sentence_model.encode(data_test['jawaban_referensi'].str.lower().values);

    y_test = data_test['score']/5

    predicts_test = [1-spatial.distance.cosine(answer_student_embed, answer_reference_embed) for answer_student_embed, answer_reference_embed in zip(data_test['answer_student_embed'].values, data_test['answer_reference_embed'].values)]

    print('Val Mean Absolute Error: {}'.format(round(mean_absolute_error(y_test, predicts_test), 4)))
    print('Val Mean Squared Error: {}'.format(round(mean_squared_error(y_test, predicts_test), 4)))
        
    del sentence_model 

Model Ongoing: bert-base-multilingual-uncased



Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.2386
Val Mean Squared Error: 0.0875
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3627
Val Mean Squared Error: 0.1996
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3273
Val Mean Squared Error: 0.1629
Model Ongoing: bert-base-multilingual-cased



Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.2469
Val Mean Squared Error: 0.0831
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3312
Val Mean Squared Error: 0.1608
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3387
Val Mean Squared Error: 0.1744
Model Ongoing: distilbert-base-multilingual-cased



Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.2748
Val Mean Squared Error: 0.1114
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3468
Val Mean Squared Error: 0.1821
Answer Student Test Embedding...
Answer Reference Test Embedding...
Val Mean Absolute Error: 0.3789
Val Mean Squared Error: 0.2174


## Fine-Tuning

### Constant

In [ ]:
MODEL_NAME_ORI = 'distilbert-base-multilingual-cased'
NUM_EPOCHS_TUNE = 1
BATCH_SIZE_TUNE = 10
EVALUATION_STEPS_TUNE = 1000

### Helper Function

In [ ]:
from typing import Union, List

class InputExample:
  """
  Structure for one input example with texts, the label and a unique id
  """
  def __init__(self, guid: str, texts: List[str], label: Union[int, float]):
    """
    Creates one InputExample with the given texts, guid and label
    str.strip() is called on both texts.
    :param guid
        id for the example
    :param texts
        the texts for the example
    :param label
        the label for the example
    """
    self.guid = guid
    self.texts = [text.strip() for text in texts]
    self.label = label
  
class DataReader:
  def __init__(self, normalize_scores=True, s1_col_name='jawaban_pelajar', 
               s2_col_name='jawaban_referensi', score_col_name='score', min_score=0, max_score=5):
    self.normalize_scores = normalize_scores
    self.score_col_name = score_col_name
    self.s1_col_name = s1_col_name
    self.s2_col_name = s2_col_name
    self.min_score = min_score
    self.max_score = max_score

  def get_examples(self, dataset, dataname, max_examples=0):
    examples = []
    for index, row in dataset.iterrows():
      score = float(row[self.score_col_name])
      if self.normalize_scores:
        score = (score - self.min_score) / (self.max_score - self.min_score)
      s1 = row[self.s1_col_name]
      s2 = row[self.s2_col_name]
      examples.append(InputExample(guid=dataname+str(index), texts=[s1, s2], label=score))

      if max_examples > 0 and len(examples) >= max_examples:
        break
    
    return examples

In [ ]:
def fine_tuning_bert(sentence_model, data_train, model_location, batch_size=BATCH_SIZE_TUNE, num_epochs=NUM_EPOCHS_TUNE, evaluation_steps=EVALUATION_STEPS_TUNE):
    data_val_bert = data_train.sample(frac=0.2, random_state=24)
    data_train_bert = data_train.drop(data_val_bert.index)

    data_reader = DataReader()
    train_data = SentencesDataset(data_reader.get_examples(data_train_bert, 'train'), model=sentence_model)
    train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model=sentence_model)

    val_data = SentencesDataset(data_reader.get_examples(data_val_bert, 'validation'), model=sentence_model)
    val_dataloader = DataLoader(val_data, shuffle=False, batch_size=batch_size)
    evaluator = EmbeddingSimilarityEvaluator(val_dataloader)
    
    warmup_steps = math.ceil(len(train_dataloader) * num_epochs / batch_size * 0.1)
    
    sentence_model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          output_path=model_location)

### Main

In [ ]:
word_embedding_model = models.Transformer(MODEL_NAME_ORI)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                              pooling_mode_mean_tokens=True,
                              pooling_mode_cls_token=False,
                              pooling_mode_max_tokens=False)

sentence_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
model_location = '{}/Code/Model/model-bert-indonesia/model-{}-mean'.format(ROOT_PATH, MODEL_NAME_ORI)
fine_tuning_bert(sentence_model, data_train, model_location)

Epoch: 100%|██████████| 1/1 [1:34:32<00:00, 5672.10s/it]
